# Description
link/ https://www.propertyfinder.eg/en/search?c=1&fu=0&ob=mr&page=3

### The data comes from the propertyfinder site

Propertyfinder is a Developer of a real estate rental marketplace designed to connect property seekers to real estate agents. The company's marketplace allows filtered searching of available options for properties to buy from apartments to villas, townhouses, penthouses, compounds, offices, and plots of land, enabling customers to find real estate properties that fit their requirements.

11200 rows, 11 cols

Numerical & Categorical

Missing Values included (not significant, small number)

Outliers included

Can be used for EDA & Visualization or Prediction

I will predict the price of the property in every governorate depend on the address ,area, size of property, number of rooms and other Vactor.
#### Sample Questions:
Is there a correlation between the city of property and the price?

Is the area effect the price more or the number of rooms? 

Which city is the most expensive in housing price?

Which property type is most expensive ?

Which company sell more in the site?

## Here is the way I get the data from the site

In [1]:
#import libraries
from selenium import webdriver
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, WebDriverException
from bs4 import BeautifulSoup
import re
import requests
from tld import get_fld
import time
import pandas as pd
import plotly.express as px
import seaborn as sns

In [109]:
#open the chrome webdriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) 
driver.implicitly_wait(3)

In [15]:
# function to get elements
def get_element(xpath, attribute=None):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.text if attribute is None else element.get_attribute(attribute)
    except:
        return "Element not found"

In [ ]:
page = 3
data1 = []

# Navigating to all pages
while page <= 450:  # Stop loop when reaching page number 450

    driver.get(f"https://www.propertyfinder.eg/en/search?c=1&fu=0&ob=mr&page={page}")
    time.sleep(2)  #  sleep time
    number_of_iterations = driver.find_elements(By.XPATH, '//*[@id="root_element"]/main/div[6]/div[1]/ul/li')
    print(f"Navigating Page number {page}")

    # Loop through each item on the page
    for i, item in enumerate(number_of_iterations, start=1):
        # Construct XPath for each element
        path = f'//*[@id="root_element"]/main/div[6]/div[1]/ul/li[{i}]'

        # Extract data for each property
        property_type = get_element(f'{path}/article/div/div/section[2]/div[1]/div[1]/p/span')
        price = get_element(f'{path}/article/div/div/section[2]/div[1]/div[1]/div/p')
        description = get_element(f'{path}/article/div/div/section[2]/div[1]/div[1]/h2')
        address = get_element(f'{path}/article/div/div/section[2]/div[2]/div[1]/p')
        bedroom_number = get_element(f'{path}/article/div/div/section[2]/div[2]/div[2]/p[1]')
        bathroom_number = get_element(f'{path}/article/div/div/section[2]/div[2]/div[2]/p[2]')
        area = get_element(f'{path}/article/div/div/section[2]/div[2]/div[2]/p[3]')
        listed_in_site=get_element(f'{path}/article/div/footer/div/div[1]/p')
        phone = get_element(f'{path}/article/div/footer/div/div[2]/div[1]/a[1]', attribute='href')
        company = get_element(f'{path}/article/div/div/section[2]/div[1]/div[2]/div/div/img[2]', attribute='title')
        link = get_element(f'{path}/article/a', attribute='href')

        # Append the row data to the main list
        data1.append([property_type, address, bedroom_number, bathroom_number, area, phone, description,listed_in_site,link,company, price])

    page += 1


In [17]:
# Specify the path to CSV file
csv_path = r'E:\Epsilon\code2\Mid Project\propertyfinder_english.csv'

# Write the header row to the CSV file
with open(csv_path, 'w',encoding='utf-8-sig',newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(["property_type", "address", "bedroom_number","bathroom_number","area" ,"phone","description","listed_in_site","link","company","price"])


In [18]:
# Open the CSV file in append mode and write the main list to the file
with open(csv_path, 'a', encoding='utf-8-sig', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerows(data1)  

In [19]:
# Close the browser window
driver.quit()

In [2]:
# read dataframe
df=pd.read_csv('E:\Epsilon\code2\Mid Project\propertyfinder_english.csv')
df.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295 sqm,tel:+201025447652,Villa For sale 295M Golf View Stone Park New C...,Listed 11 hours ago,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property logo image,"3,000,000 EGP"
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125 sqm,tel:+201016645466,A distinctive chalet for sale in Porto Marina,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate logo image,"8,500,000 EGP"
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168 sqm,tel:+201280862222,Opportunity Under Market price in zed towers,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate logo image,"11,400,000 EGP"
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185 sqm,tel:+20221208034,Apartment for sale - jasper lake new giza,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate logo image,"12,500,000 EGP"
4,Apartment,"Noor City, Cairo",3,3,131 sqm,tel:+201096566594,Noor City Direct from TMG | 131m2 Apartment,Listed 8 hours ago,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct logo image,"529,000 EGP"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11200 entries, 0 to 11199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_type    11200 non-null  object
 1   address          11200 non-null  object
 2   bedroom_number   11200 non-null  object
 3   bathroom_number  11200 non-null  object
 4   area             11200 non-null  object
 5   phone            11200 non-null  object
 6   description      11200 non-null  object
 7   listed_in_site   11200 non-null  object
 8   link             11200 non-null  object
 9   company          11200 non-null  object
 10  price            11200 non-null  object
dtypes: object(11)
memory usage: 962.6+ KB


In [4]:
# split the address column to governorate,City and address_only column
df['governorate']=df['address'].apply(lambda x: x.split(',')[-1])
df['City']=df['address'].apply(lambda x: x.split(',')[-2])
df['address_only'] = df['address'].apply(lambda x: ','.join(x.split(',')[0:-2]))

In [5]:
# get counts to property type to determine which property not important and delete it
df['property_type'].value_counts()

Apartment          5595
Villa              1743
Chalet             1466
Townhouse           957
Twin House          409
Duplex              390
Penthouse           302
iVilla              233
Hotel Apartment      26
Land                 26
Whole Building       12
Roof                 10
Compound             10
Palace                9
Full Floor            8
Cabin                 4
Name: property_type, dtype: int64

In [6]:
# the property_type Land and Whole Building return some not valued data 
# and becouse of it small rows I will delet all this row
df = df.loc[df["property_type"] != 'Land']
df = df.loc[df["property_type"] != 'Whole Building']

In [7]:
df['property_type'].value_counts()

Apartment          5595
Villa              1743
Chalet             1466
Townhouse           957
Twin House          409
Duplex              390
Penthouse           302
iVilla              233
Hotel Apartment      26
Roof                 10
Compound             10
Palace                9
Full Floor            8
Cabin                 4
Name: property_type, dtype: int64

In [8]:
# cheack if there are any string value in bedroom column
df['bedroom_number'].unique()

array(['4', '3', '2', '1', '5', '6', 'studio', '7', '7+'], dtype=object)

In [9]:
# becouse the studio has 1 room i will replace studio by 1
df['bedroom_number'] = df['bedroom_number'].replace('studio', '1')

In [10]:
df['bedroom_number'].unique()

array(['4', '3', '2', '1', '5', '6', '7', '7+'], dtype=object)

In [11]:
#replace every +7 by 7 
df['bedroom_number'] = df['bedroom_number'].replace('7+', '7')

In [12]:
# convert 'bedroom_number' the column to integer
df['bedroom_number']=df['bedroom_number'].astype(int)

In [13]:
df['bathroom_number'].unique()

array(['4', '2', '3', '1', '5', '7', '6', '7+', '67 sqm', '62 sqm',
       '108 sqm'], dtype=object)

In [14]:
# determine which row in 'bathroom_number' contian unvaild value like '67 sqm', '62 sqm','108 sqm'
df[df["bathroom_number"].isin(['62 sqm', '67 sqm', '108 sqm'])]

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price,governorate,City,address_only
4628,Apartment,"Taj City, 5th Settlement Compounds, The 5th Se...",1,67 sqm,Element not found,tel:+201098320983,Studio for sale in Taj City Compound New Cairo...,Listed 17 days ago,https://www.propertyfinder.eg/en/plp/buy/apart...,GPM Group for Property management logo image,"2,050,000 EGP",Cairo,New Cairo City,"Taj City, 5th Settlement Compounds, The 5th Se..."
5313,Apartment,"1st Avenue South side, Hurghada Resorts, Hurgh...",1,62 sqm,Element not found,tel:+201097797366,امتلك منزلك الممـــيز في الغردقة,Listed 1 month ago,https://www.propertyfinder.eg/en/plp/buy/apart...,Aka Group Real estate logo image,"2,560,000 EGP",Red Sea,Hurghada,"1st Avenue South side, Hurghada Resorts"
6072,Chalet,"North Coast Resorts, North Coast",2,108 sqm,Element not found,tel:+201101334593,Chalet for sale North coast (Aqua View Village) -,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/chale...,RE/MAX Avalon logo image,"1,900,000 EGP",North Coast,North Coast Resorts,


In [15]:
# replace this value by 1
df["bathroom_number"] = df["bathroom_number"].replace(['67 sqm', '62 sqm', '108 sqm'], 1)

In [16]:
#replace every +7 by 7 
df['bathroom_number'] = df['bathroom_number'].replace('7+', '7')

In [17]:
# convert 'bathroom_number' the column to integer
df['bathroom_number']=df['bathroom_number'].astype(int)


In [18]:
df['bathroom_number'].unique()

array([4, 2, 3, 1, 5, 7, 6])

In [19]:
# Replace 'Element not found' with ['67 sqm', '62 sqm', '108 sqm'] in the "area"  becouse it wrong orentation
df['area'] = df['area'].replace(to_replace={'Element not found': ['67 sqm', '62 sqm', '108 sqm']})


In [20]:
# delete the string value "sqm" from area column and handel commas that representing thousands.

df['area'] = df['area'].str.replace('sqm', '').str.replace(',', '').astype(int)


In [21]:
df.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price,governorate,City,address_only
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295,tel:+201025447652,Villa For sale 295M Golf View Stone Park New C...,Listed 11 hours ago,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property logo image,"3,000,000 EGP",Cairo,New Cairo City,"Stone Park, 5th Settlement Compounds, The 5th ..."
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125,tel:+201016645466,A distinctive chalet for sale in Porto Marina,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate logo image,"8,500,000 EGP",North Coast,Al Alamein,Porto Marina
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168,tel:+201280862222,Opportunity Under Market price in zed towers,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate logo image,"11,400,000 EGP",Giza,Sheikh Zayed City,"Park Side Residence, Zed Towers, Sheikh Zayed ..."
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185,tel:+20221208034,Apartment for sale - jasper lake new giza,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate logo image,"12,500,000 EGP",Giza,6 October City,"New Giza, Cairo Alexandria Desert Road"
4,Apartment,"Noor City, Cairo",3,3,131,tel:+201096566594,Noor City Direct from TMG | 131m2 Apartment,Listed 8 hours ago,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct logo image,"529,000 EGP",Cairo,Noor City,


In [22]:
# delete 'tel:+' from phone number
df["phone"]=df["phone"].str.replace('tel:+','',regex=True)

In [23]:
df["phone"] = df["phone"].str[1:]


In [24]:
df.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price,governorate,City,address_only
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295,201025447652,Villa For sale 295M Golf View Stone Park New C...,Listed 11 hours ago,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property logo image,"3,000,000 EGP",Cairo,New Cairo City,"Stone Park, 5th Settlement Compounds, The 5th ..."
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125,201016645466,A distinctive chalet for sale in Porto Marina,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate logo image,"8,500,000 EGP",North Coast,Al Alamein,Porto Marina
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168,201280862222,Opportunity Under Market price in zed towers,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate logo image,"11,400,000 EGP",Giza,Sheikh Zayed City,"Park Side Residence, Zed Towers, Sheikh Zayed ..."
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185,20221208034,Apartment for sale - jasper lake new giza,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate logo image,"12,500,000 EGP",Giza,6 October City,"New Giza, Cairo Alexandria Desert Road"
4,Apartment,"Noor City, Cairo",3,3,131,201096566594,Noor City Direct from TMG | 131m2 Apartment,Listed 8 hours ago,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct logo image,"529,000 EGP",Cairo,Noor City,


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11162 entries, 0 to 11199
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_type    11162 non-null  object
 1   address          11162 non-null  object
 2   bedroom_number   11162 non-null  int32 
 3   bathroom_number  11162 non-null  int32 
 4   area             11162 non-null  int32 
 5   phone            11162 non-null  object
 6   description      11162 non-null  object
 7   listed_in_site   11162 non-null  object
 8   link             11162 non-null  object
 9   company          11162 non-null  object
 10  price            11162 non-null  object
 11  governorate      11162 non-null  object
 12  City             11162 non-null  object
 13  address_only     11162 non-null  object
dtypes: int32(3), object(11)
memory usage: 1.1+ MB


In [26]:
# delete the some added text 
df['company']=df['company'].str.replace('logo image','')
df['company']

0                               GPR Property 
1                       LEVEL UP Real Estate 
2                     New Avenue Real Estate 
3                        Premium Real estate 
4                                 TMG Direct 
                         ...                 
11195                                The Way 
11196                 New Avenue Real Estate 
11197                       Element not found
11198    I properties Real Estate Consultant 
11199                             Smart Move 
Name: company, Length: 11162, dtype: object

In [27]:
# some price have string word 'Ask for price' ,we will count them
df[df['price']=='Ask for price'].value_counts().sum()

223

In [28]:
# I will delete all row contain 'Ask for price' because the price is important factor, and the number of this row is small
df = df[df['price'] != 'Ask for price']

In [29]:
# delet 'EGP' from price and handel commas that representing thousands and convert it to integer
df['price']=df['price'].str.replace('EGP','').str.replace(',','').astype(int)

In [30]:
# convert price to thousands
df['price']=df['price']/1000

In [31]:
df = df.rename(columns={'price': 'price_in_thousands'})


In [32]:
df.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price_in_thousands,governorate,City,address_only
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295,201025447652,Villa For sale 295M Golf View Stone Park New C...,Listed 11 hours ago,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property,3000.0,Cairo,New Cairo City,"Stone Park, 5th Settlement Compounds, The 5th ..."
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125,201016645466,A distinctive chalet for sale in Porto Marina,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate,8500.0,North Coast,Al Alamein,Porto Marina
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168,201280862222,Opportunity Under Market price in zed towers,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate,11400.0,Giza,Sheikh Zayed City,"Park Side Residence, Zed Towers, Sheikh Zayed ..."
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185,20221208034,Apartment for sale - jasper lake new giza,Listed 1 day ago,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate,12500.0,Giza,6 October City,"New Giza, Cairo Alexandria Desert Road"
4,Apartment,"Noor City, Cairo",3,3,131,201096566594,Noor City Direct from TMG | 131m2 Apartment,Listed 8 hours ago,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct,529.0,Cairo,Noor City,


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10939 entries, 0 to 11199
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   property_type       10939 non-null  object 
 1   address             10939 non-null  object 
 2   bedroom_number      10939 non-null  int32  
 3   bathroom_number     10939 non-null  int32  
 4   area                10939 non-null  int32  
 5   phone               10939 non-null  object 
 6   description         10939 non-null  object 
 7   listed_in_site      10939 non-null  object 
 8   link                10939 non-null  object 
 9   company             10939 non-null  object 
 10  price_in_thousands  10939 non-null  float64
 11  governorate         10939 non-null  object 
 12  City                10939 non-null  object 
 13  address_only        10939 non-null  object 
dtypes: float64(1), int32(3), object(10)
memory usage: 1.1+ MB


In [34]:
df.describe()

,bedroom_number,bathroom_number,area,price_in_thousands
count,10939.000000,10939.000000,10939.000000,10939.000000
mean,2.993875,2.832709,343.725843,11526.010433
std,1.056832,1.156594,9993.174183,15730.631434
min,1.000000,1.000000,3.000000,100.000000
25%,2.000000,2.000000,130.000000,4400.000000
50%,3.000000,3.000000,170.000000,7950.000000
75%,3.000000,3.000000,230.000000,13788.610000
max,7.000000,7.000000,739150.000000,798500.000000


In [35]:
df[df['price_in_thousands']==100]

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price_in_thousands,governorate,City,address_only
8529,Townhouse,"At East, Mostakbal City Compounds, Mostakbal C...",4,6,160,201129671140,امتلك وحدتك فى قلب مدينه المستقبل,Listed 8 days ago,https://www.propertyfinder.eg/en/plp/buy/townh...,New Chapter Real Estate,100.0,Cairo,Mostakbal City - Future City,"At East, Mostakbal City Compounds"
8557,Apartment,"El Motamayez District, Badr City, Cairo",3,2,140,201026255489,شقه 140 م بحرى بفيو الحزام الاخضر دور اول,Listed 2 hours ago,https://www.propertyfinder.eg/en/plp/buy/apart...,El Mamar Real Estate,100.0,Cairo,Badr City,El Motamayez District


In [36]:
df['listed_in_site']= df['listed_in_site'].str.replace('Listed','').str.replace('ago','')
df['listed_in_site'].unique()

array([' 11 hours ', ' 1 day ', ' 8 hours ', ' 7 hours ', ' 13 hours ',
       ' 6 hours ', ' 12 hours ', ' 4 hours ', ' 3 hours ', ' 2 days ',
       ' 3 days ', ' 4 days ', ' 5 days ', ' 6 days ', ' 17 hours ',
       ' 7 days ', ' 8 days ', ' 12 days ', ' 11 days ', ' 10 days ',
       ' 13 days ', ' 15 days ', ' 9 days ', ' 14 days ', ' 19 days ',
       ' 22 days ', ' 18 days ', ' 17 days ', ' 16 days ', ' 20 days ',
       ' 21 days ', ' 14 hours ', ' 24 days ', ' 26 days ', ' 29 days ',
       ' 23 days ', ' 25 days ', ' 27 days ', ' 28 days ', ' 1 month ',
       ' 2 months ', ' 3 months ', ' 5 months ', ' more than 6 months ',
       ' 4 months ', ' 6 months ', ' 19 hours ', ' 20 hours ',
       ' 15 hours ', ' 18 hours ', ' 21 hours ', ' 22 hours ',
       ' 16 hours ', ' 2 hours ', ' 23 hours ', ' 9 hours ', ' 10 hours '],
      dtype=object)

In [37]:
# split the column listed_in_site to peroid 
df['peroid']= df['listed_in_site'].str.split().str[1]

In [38]:
df['peroid'].unique()

array(['hours', 'day', 'days', 'month', 'months', 'than'], dtype=object)

In [39]:
df['peroid']=df['peroid'].str.replace('than', 'months')

In [40]:
df['peroid']=df['peroid'].str.replace('months', 'month')

In [41]:
df['peroid']=df['peroid'].str.replace('days', 'day')

In [42]:
df['peroid'].unique()

array(['hours', 'day', 'month'], dtype=object)

In [43]:
# split the column listed_in_site to the time 

df['time']= df['listed_in_site'].str.split().str[0]


In [44]:
df['time'].unique()

array(['11', '1', '8', '7', '13', '6', '12', '4', '3', '2', '5', '17',
       '10', '15', '9', '14', '19', '22', '18', '16', '20', '21', '24',
       '26', '29', '23', '25', '27', '28', 'more'], dtype=object)

In [45]:
df['time']=df['time'].str.replace('more','6')

In [46]:
df['time'].unique()

array(['11', '1', '8', '7', '13', '6', '12', '4', '3', '2', '5', '17',
       '10', '15', '9', '14', '19', '22', '18', '16', '20', '21', '24',
       '26', '29', '23', '25', '27', '28'], dtype=object)

In [47]:
df['time']=df['time'].astype(int)

In [48]:
df.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price_in_thousands,governorate,City,address_only,peroid,time
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295,201025447652,Villa For sale 295M Golf View Stone Park New C...,11 hours,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property,3000.0,Cairo,New Cairo City,"Stone Park, 5th Settlement Compounds, The 5th ...",hours,11
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125,201016645466,A distinctive chalet for sale in Porto Marina,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate,8500.0,North Coast,Al Alamein,Porto Marina,day,1
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168,201280862222,Opportunity Under Market price in zed towers,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate,11400.0,Giza,Sheikh Zayed City,"Park Side Residence, Zed Towers, Sheikh Zayed ...",day,1
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185,20221208034,Apartment for sale - jasper lake new giza,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate,12500.0,Giza,6 October City,"New Giza, Cairo Alexandria Desert Road",day,1
4,Apartment,"Noor City, Cairo",3,3,131,201096566594,Noor City Direct from TMG | 131m2 Apartment,8 hours,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct,529.0,Cairo,Noor City,,hours,8


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10939 entries, 0 to 11199
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   property_type       10939 non-null  object 
 1   address             10939 non-null  object 
 2   bedroom_number      10939 non-null  int32  
 3   bathroom_number     10939 non-null  int32  
 4   area                10939 non-null  int32  
 5   phone               10939 non-null  object 
 6   description         10939 non-null  object 
 7   listed_in_site      10939 non-null  object 
 8   link                10939 non-null  object 
 9   company             10939 non-null  object 
 10  price_in_thousands  10939 non-null  float64
 11  governorate         10939 non-null  object 
 12  City                10939 non-null  object 
 13  address_only        10939 non-null  object 
 14  peroid              10939 non-null  object 
 15  time                10939 non-null  int32  
dtypes: f

In [50]:
df['peroid'].unique()

array(['hours', 'day', 'month'], dtype=object)

In [51]:
# Define a function to convert time to hours
def convert_to_hours(row):
    if row['peroid'] == 'day':
        return row['time'] * 24
    elif row['peroid'] == 'month':
        return row['time'] *30*24
    else:
        return row['time']

In [52]:
# Apply the function to the DataFrame
df['hours'] = df.apply(convert_to_hours, axis=1)

In [53]:
df.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,price_in_thousands,governorate,City,address_only,peroid,time,hours
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295,201025447652,Villa For sale 295M Golf View Stone Park New C...,11 hours,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property,3000.0,Cairo,New Cairo City,"Stone Park, 5th Settlement Compounds, The 5th ...",hours,11,11
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125,201016645466,A distinctive chalet for sale in Porto Marina,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate,8500.0,North Coast,Al Alamein,Porto Marina,day,1,24
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168,201280862222,Opportunity Under Market price in zed towers,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate,11400.0,Giza,Sheikh Zayed City,"Park Side Residence, Zed Towers, Sheikh Zayed ...",day,1,24
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185,20221208034,Apartment for sale - jasper lake new giza,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate,12500.0,Giza,6 October City,"New Giza, Cairo Alexandria Desert Road",day,1,24
4,Apartment,"Noor City, Cairo",3,3,131,201096566594,Noor City Direct from TMG | 131m2 Apartment,8 hours,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct,529.0,Cairo,Noor City,,hours,8,8


Do some EDA and visualization

In [74]:
# determain the outliers
def get_outliers(column):
    q1=column.quantile(0.25)
    q3=column.quantile(0.75)
    iqr=q3-q1
    upper= q3 + 1.5*iqr
    lower= q1 - 1.5*iqr

    return column[(column< lower)| (column> upper)].index

In [76]:
# There are outliers values in the price, that came from the Inconsistency in the types of property
px.box(df, x='price_in_thousands')

In [77]:
# the figure below show how the Inconsistency of the property price , 
# price of villa and apartment is too hight and the price of hotel apartment and roof is to low ,
# that came from the area of this property
px.histogram(df,x='price_in_thousands',y='property_type')

In [78]:
outliers_index=get_outliers(df['price_in_thousands'])
outliers_index

Int64Index([    7,     8,    16,    37,    43,    46,    61,    94,   100,
              107,
            ...
            11095, 11107, 11142, 11158, 11162, 11166, 11169, 11177, 11179,
            11196],
           dtype='int64', length=740)

In [ ]:
# becouse of the Inconsistency in price that came from the area I will made a new column the price of squre in meter ( price/area)
df['price_of_squre_meter_in_thousands']= df['price_in_thousands']/df['area']

In [56]:
df['price_of_squre_meter_in_thousands'].mean()

55.78511085382483

In [57]:
px.histogram(df,x='price_of_squre_meter_in_thousands')

In [58]:
px.box(df, x='price_of_squre_meter_in_thousands')
# it's obviously there are outlier in this feature 

In [60]:
outliers_index=get_outliers(df['price_of_squre_meter_in_thousands'])
outliers_index


Int64Index([  100,   113,   129,   130,   134,   137,   178,   189,   206,
              211,
            ...
            10952, 10985, 10986, 11007, 11014, 11094, 11140, 11162, 11169,
            11185],
           dtype='int64', length=264)

In [61]:
len(outliers_index)
# the len of outliers not much 

264

In [62]:
# we will replace this outliers with the median
df['price_of_squre_meter_in_thousands'].loc[outliers_index] = df['price_of_squre_meter_in_thousands'].median()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_5660\1357419006.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [63]:
px.box(df, x='price_of_squre_meter_in_thousands')

In [72]:
df['price_of_squre_meter_in_thousands'].mean()

51.29415763507343

In [64]:
px.histogram(df,x='price_of_squre_meter_in_thousands')

Is there a correlation between the governorate of property and the price of meter?


In [84]:
df['governorate'].unique()

array([' Cairo', ' North Coast', ' Giza', ' Suez', ' South Sainai',
       ' Red Sea', ' Alexandria', ' Al Daqahlya', ' Demyat', ' Qalyubia',
       ' Matrouh', ' Qena', ' Port Saeed'], dtype=object)

In [88]:
# Convert 'governorate' column to one-hot encoding
df_encoded = pd.get_dummies(df, columns=['governorate'], drop_first=True)
df_encoded.head()

,property_type,address,bedroom_number,bathroom_number,area,phone,description,listed_in_site,link,company,...,governorate_ Demyat,governorate_ Giza,governorate_ Matrouh,governorate_ North Coast,governorate_ Port Saeed,governorate_ Qalyubia,governorate_ Qena,governorate_ Red Sea,governorate_ South Sainai,governorate_ Suez
0,Villa,"Stone Park, 5th Settlement Compounds, The 5th ...",4,4,295,201025447652,Villa For sale 295M Golf View Stone Park New C...,11 hours,https://www.propertyfinder.eg/en/plp/buy/villa...,GPR Property,...,0,0,0,0,0,0,0,0,0,0
1,Apartment,"Porto Marina, Al Alamein, North Coast",3,2,125,201016645466,A distinctive chalet for sale in Porto Marina,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,LEVEL UP Real Estate,...,0,0,0,1,0,0,0,0,0,0
2,Apartment,"Park Side Residence, Zed Towers, Sheikh Zayed ...",3,3,168,201280862222,Opportunity Under Market price in zed towers,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,New Avenue Real Estate,...,0,1,0,0,0,0,0,0,0,0
3,Apartment,"New Giza, Cairo Alexandria Desert Road, 6 Octo...",3,2,185,20221208034,Apartment for sale - jasper lake new giza,1 day,https://www.propertyfinder.eg/en/plp/buy/apart...,Premium Real estate,...,0,1,0,0,0,0,0,0,0,0
4,Apartment,"Noor City, Cairo",3,3,131,201096566594,Noor City Direct from TMG | 131m2 Apartment,8 hours,https://www.propertyfinder.eg/en/plp/buy/apart...,TMG Direct,...,0,0,0,0,0,0,0,0,0,0


In [105]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10939 entries, 0 to 11199
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   property_type                      10939 non-null  object 
 1   address                            10939 non-null  object 
 2   bedroom_number                     10939 non-null  int32  
 3   bathroom_number                    10939 non-null  int32  
 4   area                               10939 non-null  int32  
 5   phone                              10939 non-null  object 
 6   description                        10939 non-null  object 
 7   listed_in_site                     10939 non-null  object 
 8   link                               10939 non-null  object 
 9   company                            10939 non-null  object 
 10  price_in_thousands                 10939 non-null  float64
 11  City                               10939 non-null  obj

In [106]:
# Calculate correlation between all governorate columns and 'price in meter'
city_columns = df_encoded.columns[df_encoded.columns.str.startswith('governorate_')]
correlation = df_encoded[city_columns].corrwith(df_encoded['price_of_squre_meter_in_thousands'])
correlation

governorate_ Alexandria     -0.069556
governorate_ Cairo          -0.103594
governorate_ Demyat         -0.032149
governorate_ Giza           -0.020160
governorate_ Matrouh        -0.021670
governorate_ North Coast     0.204873
governorate_ Port Saeed     -0.008669
governorate_ Qalyubia       -0.058706
governorate_ Qena           -0.006375
governorate_ Red Sea         0.028656
governorate_ South Sainai   -0.027451
governorate_ Suez           -0.017065
dtype: float64

In [107]:
df_encoded.corr()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_5660\3451755271.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,bedroom_number,bathroom_number,area,price_in_thousands,time,hours,price_of_squre_meter_in_thousands,governorate_ Alexandria,governorate_ Cairo,governorate_ Demyat,governorate_ Giza,governorate_ Matrouh,governorate_ North Coast,governorate_ Port Saeed,governorate_ Qalyubia,governorate_ Qena,governorate_ Red Sea,governorate_ South Sainai,governorate_ Suez
bedroom_number,1.000000,0.811291,-0.001295,0.414313,-0.004549,0.042605,-0.006574,0.017715,0.090384,0.014487,0.083284,-0.022032,-0.092153,-0.012718,0.043560,0.000055,-0.196257,-0.033667,-0.081930
bathroom_number,0.811291,1.000000,0.001486,0.454346,0.019372,-0.002000,0.100215,-0.049149,0.083633,-0.006086,0.119923,-0.026995,-0.097682,-0.015583,0.034179,-0.006884,-0.167959,-0.036829,-0.125752
area,-0.001295,0.001486,1.000000,0.003398,-0.013071,-0.006951,-0.023302,-0.001290,-0.011928,-0.000322,0.022473,-0.000634,-0.006870,-0.000337,-0.000325,-0.000099,-0.004464,-0.000854,-0.004160
price_in_thousands,0.414313,0.454346,0.003398,1.000000,0.005543,-0.045360,0.392948,-0.026092,-0.002953,-0.014047,0.016035,-0.014201,0.060479,-0.007303,-0.021334,-0.002447,-0.049361,-0.017608,-0.063140
time,-0.004549,0.019372,-0.013071,0.005543,1.000000,0.003601,0.030653,-0.012642,0.009013,-0.020175,0.021185,-0.011593,-0.035927,0.002508,-0.018609,-0.005688,-0.011940,-0.014850,0.022410
hours,0.042605,-0.002000,-0.006951,-0.045360,0.003601,1.000000,-0.153398,0.042928,-0.079771,0.015235,0.049764,0.015455,-0.050351,0.030427,0.134989,-0.004815,0.089330,0.014129,0.020889
price_of_squre_meter_in_thousands,-0.006574,0.100215,-0.023302,0.392948,0.030653,-0.153398,1.000000,-0.069556,-0.103594,-0.032149,-0.020160,-0.021670,0.204873,-0.008669,-0.058706,-0.006375,0.028656,-0.027451,-0.017065
governorate_ Alexandria,0.017715,-0.049149,-0.001290,-0.026092,-0.012642,0.042928,-0.069556,1.000000,-0.100553,-0.003252,-0.069409,-0.002519,-0.044306,-0.001454,-0.005824,-0.001028,-0.021879,-0.003849,-0.020918
governorate_ Cairo,0.090384,0.083633,-0.011928,-0.002953,0.009013,-0.079771,-0.103594,-0.100553,1.000000,-0.028291,-0.603797,-0.021910,-0.385417,-0.012647,-0.050659,-0.008943,-0.190328,-0.033480,-0.181971
governorate_ Demyat,0.014487,-0.006086,-0.000322,-0.014047,-0.020175,0.015235,-0.032149,-0.003252,-0.028291,1.000000,-0.019528,-0.000709,-0.012465,-0.000409,-0.001638,-0.000289,-0.006156,-0.001083,-0.005885


Is the area effect the price more or the number of rooms? 


In [109]:
df.corr()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_5660\1134722465.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,bedroom_number,bathroom_number,area,price_in_thousands,time,hours,price_of_squre_meter_in_thousands
bedroom_number,1.000000,0.811291,-0.001295,0.414313,-0.004549,0.042605,-0.006574
bathroom_number,0.811291,1.000000,0.001486,0.454346,0.019372,-0.002000,0.100215
area,-0.001295,0.001486,1.000000,0.003398,-0.013071,-0.006951,-0.023302
price_in_thousands,0.414313,0.454346,0.003398,1.000000,0.005543,-0.045360,0.392948
time,-0.004549,0.019372,-0.013071,0.005543,1.000000,0.003601,0.030653
hours,0.042605,-0.002000,-0.006951,-0.045360,0.003601,1.000000,-0.153398
price_of_squre_meter_in_thousands,-0.006574,0.100215,-0.023302,0.392948,0.030653,-0.153398,1.000000


In [110]:
px.imshow(df.corr())
# it seem that the number of bedroom effect more than the area in the price 

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_5660\2088196402.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Which governorate is the most expensive in housing price?


In [111]:
average_price_by_governorate = df.groupby('governorate')['price_of_squre_meter_in_thousands'].mean()
most_expensive_city = average_price_by_governorate.idxmax()  # Get the index (governorate name) with the maximum average price
print(f"The most expensive city in terms of housing price is: {most_expensive_city}")

The most expensive city in terms of housing price is:  North Coast


In [113]:
average_price_by_city = df.groupby('governorate')['price_of_squre_meter_in_thousands'].mean()
# Sort the cities based on average price in descending order
sorted_cities = average_price_by_city.sort_values(ascending=False)
print("Cities ordered by average housing price:")
print(sorted_cities)

Cities ordered by average housing price:
governorate
 North Coast     66.239938
 Red Sea         55.527446
 Giza            50.355366
 Suez            48.657446
 Cairo           47.964265
 Port Saeed      32.021212
 Alexandria      31.844954
 Qena            31.250000
 South Sainai    28.241111
 Matrouh         23.485389
 Demyat          19.342722
 Qalyubia        18.711285
 Al Daqahlya     18.288400
Name: price_of_squre_meter_in_thousands, dtype: float64


In [112]:
price_range_by_city = df.groupby('governorate')['price_of_squre_meter_in_thousands'].agg(['min', 'max'])
print("Price range for each city:")
print(price_range_by_city)

Price range for each city:
                     min         max
governorate                         
 Al Daqahlya    3.243243   28.000000
 Alexandria     7.500000  118.181818
 Cairo          0.625000  141.573684
 Demyat        12.422360   30.000000
 Giza           0.006332  141.573684
 Matrouh        7.750000   43.103448
 North Coast    1.136364  141.722628
 Port Saeed    31.022222   33.020202
 Qalyubia       5.373134   42.000000
 Qena          31.250000   31.250000
 Red Sea        1.012500  138.172414
 South Sainai   3.990476   85.714286
 Suez           2.241379  139.500000


In [129]:
average_price_by_governorate = df.groupby('governorate')['price_of_squre_meter_in_thousands'].mean().reset_index()

fig = px.bar(average_price_by_governorate, x='governorate', y='price_of_squre_meter_in_thousands', 
             title='Average Price of Square Meters by Governorate')

fig.show()


Which property type is most expensive ?


In [114]:
average_price_by_property_type = df.groupby('property_type')['price_of_squre_meter_in_thousands'].mean()
most_expensive_property_type = average_price_by_property_type.idxmax()  # Get the index (property type) with the maximum average price
print(f"The most expensive property type is: {most_expensive_property_type}")

The most expensive property type is: Townhouse


In [115]:
average_price_by_property_type = df.groupby('property_type')['price_of_squre_meter_in_thousands'].mean()
# Sort the property types based on average price in descending order
sorted_property_types = average_price_by_property_type.sort_values(ascending=False)
print("Property types ordered by average housing price:")
print(sorted_property_types)

Property types ordered by average housing price:
property_type
Townhouse          65.712367
Chalet             61.817605
Twin House         61.733003
Hotel Apartment    57.406401
Villa              56.100779
Penthouse          55.511909
Palace             54.477343
Cabin              49.382716
Apartment          44.789230
Duplex             42.489332
iVilla             42.319252
Compound           42.303028
Full Floor         26.500951
Roof               20.284561
Name: price_of_squre_meter_in_thousands, dtype: float64


In [131]:
average_price_by_property_type = df.groupby('property_type')['price_of_squre_meter_in_thousands'].mean().reset_index()
fig = px.bar(average_price_by_property_type, x='property_type', y='price_of_squre_meter_in_thousands', 
             title='Average Price of Square Meters by Property Type')
fig.show()

Which company sell more in the site?

In [118]:
company_sales_counts = df['company'].value_counts()
# Sort the companies based on the number of listings in descending order
sorted_companies = company_sales_counts.sort_values(ascending=False)
print("Companies ordered by number of listings:")
print(sorted_companies)

Companies ordered by number of listings:
Element not found                 1543
Egypt Best Properties West         956
TAI - The Address Investments      524
Egypt Best Properties              299
New Avenue Real Estate             188
                                  ... 
SIAG Developments                    1
ST.John Real Estate                  1
El-Kaddy Group                       1
EVIČKA                               1
Remax 360                            1
Name: company, Length: 563, dtype: int64
